In [1]:
# !pip install wget

In [2]:
import os
import shutil
import wget
import pandas as pd
from tqdm.notebook import tqdm
from astropy.table import Table
from astropy.coordinates import SkyCoord, match_coordinates_sky
import astropy.units as u

In [3]:
pddf_tess = pd.read_csv("https://archive.stsci.edu/hlsps/tess-svc/hlsp_tess-svc_tess_lcf_all-s0001-s0026_tess_v1.0_cat.csv", index_col=0)
pddf_tess = pddf_tess.loc[:,["ra","dec"]]
pddf_tess.index.name = "tess_id"
df_tess = Table.from_pandas(pddf_tess.loc[:,["ra","dec"]])
coords_tess = SkyCoord(ra=df_tess['ra']*u.degree, dec=df_tess['dec']*u.degree)

os.mkdir("radeccsvs/")
os.mkdir("xmatchcsvs/")

In [4]:
# urls = ["https://zenodo.org/records/14155156/files/24_23_prediction_xgb_dnn_fields.zip"]
urls = []
for i in range(24):
    if i==16:
        print("SOME ERROR WITH 17_16, so skipping that")
        continue
    url = f"https://zenodo.org/records/14155156/files/{(i+1):02d}_{i:02d}_prediction_xgb_dnn_fields.zip"
    urls.append(url)

In [5]:
for url in tqdm(urls, desc="URLs"):
    fn = wget.download(url)
    shutil.unpack_archive(fn, ".")
    os.remove(fn)
    dirname = fn.split(".zip")[0]

    for file in tqdm(os.listdir(dirname), desc="Reading and saving RADEC"):
        pddf_ztf = pd.read_csv(os.path.join(dirname,file), index_col=0)
        pddf_ztf = pddf_ztf.loc[:,["ra","dec"]]
        pddf_ztf.index.name = "ztf_id"
        pddf_ztf.to_csv(os.path.join("radeccsvs",file))

    shutil.rmtree(dirname)

In [6]:
for file in tqdm(os.listdir("radeccsvs"), desc="x-match"):
    pddf_ztf = pd.read_csv(os.path.join("radeccsvs",file), index_col=0)
    df_ztf = Table.from_pandas(pddf_ztf.loc[:,["ra","dec"]])
    coords_ztf = SkyCoord(ra=df_ztf['ra']*u.degree, dec=df_ztf['dec']*u.degree)
    # Perform the cross-match
    idx, d2d, d3d = match_coordinates_sky(coords_tess, coords_ztf)
    # Set a maximum separation threshold (e.g., 2 arcseconds)
    max_sep = 30 * u.arcsec
    mask = d2d < max_sep

    matched_data = pd.DataFrame({
        'ztf_id': pddf_ztf.index[idx[mask]],
        'tess_id': pddf_tess.index[mask],
        'ztf_ra': pddf_ztf.loc[:, 'ra'].values[idx[mask]],
        'tess_ra': pddf_tess.loc[:, 'ra'].values[mask],
        'ztf_dec': pddf_ztf.loc[:, 'dec'].values[idx[mask]],
        'tess_dec': pddf_tess.loc[:, 'dec'].values[mask],
        'separation_arcsec': d2d[mask].to_value(u.arcsec)
    })
    
    matched_data.to_csv(os.path.join("xmatchcsvs",file),index=False)

x-match:   0%|          | 0/204 [00:00<?, ?it/s]